<a href="https://colab.research.google.com/github/venkatanadikatla/pytorch/blob/main/RNN_with_SST_Better_Accuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pip uninstall -y torchtext
!pip install torchtext==0.6.0

Found existing installation: torchtext 0.6.0
Uninstalling torchtext-0.6.0:
  Successfully uninstalled torchtext-0.6.0
  Using cached torchtext-0.6.0-py3-none-any.whl (64 kB)


In [1]:
import copy # this module provides functions to duplicate objects. It seems to be imported but not yet used in the code
import torch # The MAIN PyTorch package
from torch import nn # contains the essential modules for building NN in pytorch.
from torch import optim # Provides optimization algorithms, such as SGD, Adam, etc
import torchtext # A library for text processing that works well with pytorch (Currently a version 0.6.0 is being used in this code)
from torchtext import data # A module in torchtext used for data handling
from torchtext import datasets # provides datasets, including various NLP datasets.

TEXT = data.Field(sequential=True, batch_first=True, lower=True) # Sequential = True indicates that the data consists of sequences.
#Batch_first=True Ensure that batch dimenstion is the first dimension in the tensor. # lower=True Converts all the text to lowercase

LABEL = data.LabelField() # A subclass of Field specifically for handling labels in a classification task.

# load data splits
train_data, val_data, test_data = datasets.SST.splits(TEXT, LABEL) #datasets.SST.splits - Loads the Standford Sentiment Treebank(SST) dataset and splits the dataset

# build dictionary
# build_vocab: Creates a mapping from tokens(words) to indices. This is essential for converting text data into numerical form that can be used by NN.
TEXT.build_vocab(train_data) # Builds the vocabulary for the text field using the training data.
LABEL.build_vocab(train_data)# Builds the vocabulary for the label field using the training data.

# hyperparameters
vocab_size = len(TEXT.vocab) # the size of the vocabulary (number of unique tokens in the training data)
label_size = len(LABEL.vocab) # the number of unique labels (classes) in the traning data
padding_idx = TEXT.vocab.stoi['<pad>'] # The index used for padding sequences to the same length
embedding_dim = 300  # The size of the word embeddings (dense vector representation of words)
hidden_dim = 256 # Size of the hidden layers in the model
dropout = 0.5
num_layers = 2
# patience =3

# build iterators
train_iter, val_iter, test_iter = data.BucketIterator.splits(
    (train_data, val_data, test_data),
    batch_size=64)

# Data.bucketiterator.splits - Creates iterators for the training, validation and test sets.


In [2]:
# # Print a few examples from the raw training data
# for i in range(50):
#     print(vars(train_data.examples[i]))
# import torch.nn.functional as F

In [3]:
# # Adding attention mechanism here to help the model focus on different part of the input sequence when making predictions.
# class Attention(nn.Module):
#     def __init__(self, hidden_dim):
#         super(Attention, self).__init__()
#         self.hidden_dim = hidden_dim
#         self.attn = nn.Linear(hidden_dim * 2, hidden_dim)
#         self.v = nn.Parameter(torch.rand(hidden_dim))

#     def forward(self, hidden, encoder_outputs):
#         timestep = encoder_outputs.size(1)
#         # h = hidden.repeat(timestep, 1, 1).transpose(0, 1)
#         h = hidden.unsqueeze(1).repeat(1, timestep, 1)
#         print(f'h shape: {h.shape}')
#         attn_energies = self.score(h, encoder_outputs)
#         return F.functional.softmax(attn_energies, dim=1)

#     def score(self, hidden, encoder_outputs):
#         energy = torch.tanh(self.attn(torch.cat([hidden, encoder_outputs], 2)))
#         energy = energy.transpose(2, 1)
#         v = self.v.repeat(encoder_outputs.size(0), 1).unsqueeze(1)
#         energy = torch.bmm(v, energy)
#         return energy.squeeze(1)




In [4]:
class RNN (torch.nn.Module):
  def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, padding_idx,dropout, num_layers=2):
    super().__init__()
    self.vocab_size = vocab_size
    self.embedding_dim = embedding_dim
    self.hidden_dim = hidden_dim
    self.label_size = label_size
    self.num_layers = num_layers
    self.dropout = nn.Dropout(dropout)
    self.padding_idx = padding_idx
    self.embedding = torch.nn.Embedding(vocab_size, embedding_dim,padding_idx = padding_idx)
    # self.rnn = torch.nn.RNN(embedding_dim,hidden_dim, nonlinearity='relu',batch_first=True)
    self.lstm = torch.nn.LSTM(embedding_dim,hidden_dim, num_layers=num_layers,dropout=dropout, batch_first=True, bidirectional=True)
    # self.attention = Attention(hidden_dim)
    self.fc = torch.nn.Linear(hidden_dim*2, output_dim) #hidden *2 because of BRNN


  def zero_state(self, batch_size):
    # Implement the function, which returns an initial hidden state.
    return (torch.zeros(self.num_layers*2, batch_size, self.hidden_dim),
            torch.zeros(self.num_layers*2, batch_size, self.hidden_dim))

  def forward(self, text):
    embedded = self.embedding(text)
    embedded = self.dropout(embedded)
    batch_size = text.size(0)
    h_0, c_0 = self.zero_state(batch_size)
    h_0, c_0 = h_0.to(text.device), c_0.to(text.device)  # Ensure the hidden state is on the same device as the input
    output, (hidden,cell) = self.lstm(embedded, (h_0, c_0))
    # Check dimensions
    # print(f'lstm_output shape: {output.shape}')  # [batch_size, seq_len, hidden_dim * 2]
    # print(f'hidden shape: {hidden.shape}')  # [num_layers * 2, batch_size, hidden_dim]


    # hidden = hidden[-1]
    hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
    # print(f'concatenated hidden shape: {hidden.shape}')  # [batch_size, hidden_dim * 2]
    # attn_weights = self.attention(hidden[-1], output)
    # context = attn_weights.unsqueeze(1).bmm(output).squeeze(1)
    # Check dimensions
    # print(f'context shape: {context.shape}')  # [batch_size, hidden_dim * 2]

    return self.fc(self.dropout(hidden))





In [5]:
def train_model(model,train_iter, optimizer, criterion, num_epochs =20):
  model.train()
  # best_val_loss = float('inf')
  # patience_counter = 0
  for epoch in range(num_epochs):
    epoch_loss = 0
    epoch_acc = 0
    correct = 0
    total = 0
    for batch in train_iter:
      optimizer.zero_grad()
      text, labels = batch.text, batch.label
      output = model(text)
      loss = criterion(output, labels)
      loss.backward()
      optimizer.step()
      epoch_loss += loss.item()
      epoch_acc += (output.argmax(1) ==labels).sum().item()

      _, predicted = torch.max(output.data,1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

    avg_loss = epoch_loss/len(train_iter)
    avg_acc = epoch_acc/len(train_iter.dataset)

    epoch_accuracy = 100*correct/total


    print(f' Epoch {epoch+1}, Train Loss: {avg_loss}, Train Accuracy: {epoch_accuracy}')

    #Early Stopping
    # val_loss = eval_model(model, val_iter, criterion)
    # if val_loss < best_val_loss:
    #   best_val_loss = val_loss
    #   patience_counter = 0

    # else:
    #   patience_counter +=1

    # if patience_counter >= patience:
    #   print('Early Stopping')
    #   break



In [6]:
def eval_model(model, val_iter, criterion):
  model.eval()
  epoch_loss = 0
  epoch_acc = 0
  correct = 0
  total = 0
  for batch in val_iter:
    text, labels = batch.text, batch.label
    output = model(text)
    loss = criterion(output, labels)
    epoch_loss +=loss.item()
    epoch_acc += (output.argmax(1)==labels).sum().item()
    _, predicted = torch.max(output.data,1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  avg_loss = epoch_loss/len(val_iter)
  avg_acc = epoch_acc/len(val_iter.dataset)

  epoch_accuracy = 100*avg_acc

  print(f'Validation Loss: {avg_loss},  Validation Accuracy: {epoch_accuracy}')
  return avg_loss




In [7]:
def test_model(model, test_iter, criterion):
  model.eval()
  epoch_loss = 0
  epoch_acc = 0
  correct = 0
  total = 0
  for batch in test_iter:
    text, labels = batch.text, batch.label
    output = model(text)
    loss = criterion(output, labels)
    epoch_loss +=loss.item()
    epoch_acc += (output.argmax(1)==labels).sum().item()
    _, predicted = torch.max(output.data,1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  avg_loss = epoch_loss/len(test_iter)
  avg_acc = epoch_acc/len(test_iter.dataset)

  epoch_accuracy = 100*correct/total

  print(f'Test Loss: {avg_loss},  Test Accuracy: {epoch_accuracy}')




In [13]:
torch.manual_seed(42)
# model = RNN(vocab_size,embedding_dim,hidden_dim,label_size, padding_idx, )
model = RNN(vocab_size, embedding_dim, hidden_dim, label_size, padding_idx,dropout, num_layers)
# optimizer = optim.Adam(model.parameters(), lr=0.0005)
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
# optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
# optimizer = optim.RMSprop(model.parameters(), lr=0.0001, alpha=0.9, weight_decay=1e-4)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
# optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
# optimizer = optim.Adagrad(model.parameters(), lr=0.01)
# optimizer = optim.Adadelta(model.parameters(), lr=1.0, rho=0.9)
# optimizer = optim.Adamax(model.parameters(), lr=0.002)
# optimizer = optim.NAdam(model.parameters(), lr=0.002)
optimizer = optim.RMSprop(model.parameters(), lr=0.001, alpha=0.9)


criterion = nn.CrossEntropyLoss()

# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
# scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=1e-5, max_lr=1e-3, step_size_up=2000, mode='triangular')

In [17]:
train_model(model,train_iter, optimizer, criterion, num_epochs =20)

 Epoch 1, Train Loss: 0.2743619574762102, Train Accuracy: 89.65355805243446
 Epoch 2, Train Loss: 0.25635228767546253, Train Accuracy: 90.22705992509363
 Epoch 3, Train Loss: 0.23135716846184945, Train Accuracy: 91.52621722846442
 Epoch 4, Train Loss: 0.23064120747705005, Train Accuracy: 91.2687265917603
 Epoch 5, Train Loss: 0.226362555107074, Train Accuracy: 91.52621722846442
 Epoch 6, Train Loss: 0.21409894842932473, Train Accuracy: 92.38061797752809
 Epoch 7, Train Loss: 0.20396954285453506, Train Accuracy: 92.7434456928839
 Epoch 8, Train Loss: 0.19999760111321263, Train Accuracy: 92.4625468164794
 Epoch 9, Train Loss: 0.18111051352166418, Train Accuracy: 93.64466292134831
 Epoch 10, Train Loss: 0.18391204875574182, Train Accuracy: 93.43398876404494
 Epoch 11, Train Loss: 0.17165971922674286, Train Accuracy: 94.00749063670412
 Epoch 12, Train Loss: 0.1633296931134676, Train Accuracy: 94.03089887640449
 Epoch 13, Train Loss: 0.15320139158683926, Train Accuracy: 94.45224719101124
 E

In [18]:
# Evaluate the model on validation data
eval_model(model, val_iter, criterion)



Validation Loss: 2.596931835015615,  Validation Accuracy: 58.58310626702997


2.596931835015615

In [19]:
# Evaluate the model on test data
test_model (model, test_iter, criterion)

Test Loss: 2.316168757847377,  Test Accuracy: 61.8552036199095


In [ ]:
test_data[43]

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
def get_specific_data_point(data, index):
    return data[index]

# Evaluate a specific data point
def evaluate_specific_data_point(model, data_point, text_field, label_field, criterion):
    model.eval()

    # Convert the data point to a tensor
    text = text_field.process([data_point.text]).to(device)
    label = torch.tensor([label_field.vocab.stoi[data_point.label]]).to(device)

    with torch.no_grad():
        output = model(text)
        loss = criterion(output, label)

        _, predicted = torch.max(output.data, 1)
        is_correct = (predicted.item() == label.item())

        print(f'Predicted: {label_field.vocab.itos[predicted.item()]}')
        print(f'Actual: {data_point.label}')
        print(f'Loss: {loss.item()}')
        print(f'Correct: {is_correct}')

# Get the specific data point
specific_data_point = get_specific_data_point(test_data, 2000)

# Evaluate the specific data point
evaluate_specific_data_point(model, specific_data_point, TEXT, LABEL, criterion)

Predicted: negative
Actual: negative
Loss: 0.001959072658792138
Correct: True


Used the following hyperparameters to increase the accuracy:

1. Changed the embedding_dim and hidden_dim to 256 (dense vector representation of words) check if this simple change in hyperparameters could change the accuracy. However, test accuracy @ 56% was little less than my baseline model accuracy @ 58%
2. With the addition of item#1 added the dropout regularization technique @0.3 to see if this can help increasing the accuracy. Doing this reduce the training accuracy to 83% and improved the test accuracyt to 57%, however, this is less than my baseline model accuracy.
3. Now adding the LSTM to 2 layers, Increasing the learning rate to 0.0001 and removing the dropout all together to see if this will increase the acccuracy - Performance is not better than item#2 infact the test accuracy dropped to 52%.
4. Now adding the LSTM to 2 layers, Increasing the learning rate to 0.0001 and adding the dropout (0.3) to see if this will increase the acccuracy - Apparently, there was no change in test accuracy. Reached test accuracy @ 51%.
5. Updated the embedding dimension to 512 and hidden layers to 512, batch_size=128 and dropout =0.3 and lr=0.0001 - Still testing accuracy is not better than baseline model. Poor accuracy 53%
6. Updating the batchsize =64 and increasing number of epochs =20 and leaving the same hyperparameters as item#5 other than batchsize and epochs for this attempt as wel - didn't perform any better - test accuracy is 53%
7. Used BRNN LSTM and dropout rate and Adam optimizer weight decay - didn't help the model test performance, reached only 51% accuracy.
8. Used BRNN LSTM and dropout ratge with SGD(optim.SGD(model.parameters(), lr=0.001, momentum=0.9)) - poor model test accuracy @ 42%
9. Used BRNN LSTM and dropout rate 0.5 with optim.RMSprop(model.parameters(), lr=0.001, alpha=0.9) - poor model test accuracy @ 57% but less than baseline model accuracy.
10. Used BRNN LSTM and dropout rate 0.5 with optim.RMSprop(model.parameters(), lr=0.0001, alpha=0.9), reduced the hidden_layers to 256 and raised epochs to 20 - ***Got the better accuracy than the base line model - Test accuracy 59%***
11. Used BRNN LSTM, removed dropout with optim.RMSprop(model.parameters(), lr=0.0001, alpha=0.9), reduced the hidden_layers to 256 and raised epochs to 20 - ***Got the better accuracy than the base line model - Test accuracy 59.5%***
12. Used BRNN LSTM, removed dropout with optim.RMSprop(model.parameters(), lr=0.0001, alpha=0.9, weight_decay=1e-4), reduced the hidden_layers to 256 and raised epochs to 20 - ***Got the poor accuracy than the base line model - Test accuracy 53.5%***
13. Used BRNN LSTM, added dropout with optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4), reduced the hidden_layers to 256 and raised epochs to 20 - ***Got the same accuracy as base line model - test accuracy 58.9%***
14. Used BRNN LSTM, added dropout with optimizer = optim.Adagrad(model.parameters(), lr=0.01), reduced the hidden_layers to 256 and raised epochs to 20 - ***Got the  accuracy less than base line model - test accuracy of 57%***
15. Used BRNN LSTM, added dropout with optimizer = optim.Adadelta(model.parameters(), lr=1.0, rho=0.9) , reduced the hidden_layers to 256 and raised epochs to 20 - ***Got the  same accuracy as base line model - test accuracy of 58.2%***
16. Used BRNN LSTM, added dropout with optimizer = optim.Adamax(model.parameters(), lr=0.002) , reduced the hidden_layers to 256 and raised epochs to 20 - ***Got the accuracy less than base line model - test accuracy of 56.7%***
17.  Used BRNN LSTM, added dropout with optimizer = optim.NAdam(model.parameters(), lr=0.002) , reduced the hidden_layers to 256 and reduced epochs to 10 - ***Got the accuracy less than base line model - test accuracy of 57.8%***
18. Used BRNN LSTM, added dropout with optim.RMSprop(model.parameters(), lr=0.0001, alpha=0.9, weight_decay=1e-4) and in addition added early stoping regularization technique , reduced the hidden_layers to 256 and  epochs to 20 - ***Got the accuracy less than base line model - test accuracy of 57.2%***



In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torchtext
from torchtext import data
from torchtext import datasets

class ResRNNCell(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(ResRNNCell, self).__init__()
        self.rnn = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.hidden_dim = hidden_dim

    def forward(self, x, hidden):
        output, new_hidden = self.rnn(x, hidden)
        if x.size(2) == self.hidden_dim:
            output = output + x
        return output, new_hidden

class StochasticDepthLayer(nn.Module):
    def __init__(self, layer, drop_prob):
        super(StochasticDepthLayer, self).__init__()
        self.layer = layer
        self.drop_prob = drop_prob

    def forward(self, x, hidden):
        if self.training and torch.rand(1).item() < self.drop_prob:
            return x, hidden
        return self.layer(x, hidden)

class ResRNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, padding_idx, dropout, num_layers=2, drop_prob=0.2):
        super(ResRNN, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers
        self.drop_prob = drop_prob
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=padding_idx)
        self.layers = nn.ModuleList()
        self.layers.append(StochasticDepthLayer(ResRNNCell(embedding_dim, hidden_dim), drop_prob))
        for _ in range(1, num_layers):
            self.layers.append(StochasticDepthLayer(ResRNNCell(hidden_dim, hidden_dim), drop_prob))
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def zero_state(self, batch_size, device):
        return (torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(device),
                torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(device))

    def forward(self, text):
        embedded = self.embedding(text)
        embedded = self.dropout(embedded)  # Apply dropout to the input embeddings
        batch_size = text.size(0)
        hidden = self.zero_state(batch_size, text.device)
        for layer in self.layers:
            embedded, hidden = layer(embedded, hidden)
        output = embedded[:, -1, :]  # Use the last hidden state
        return self.fc(self.dropout(output))

# Training, Evaluation, and Testing Functions
def train_model(model, train_iter, val_iter, optimizer, criterion, num_epochs=20, patience=3):
    best_val_loss = float('inf')
    patience_counter = 0

    model.train()
    for epoch in range(num_epochs):
        epoch_loss = 0
        epoch_acc = 0
        correct = 0
        total = 0
        for batch in train_iter:
            optimizer.zero_grad()
            text, labels = batch.text, batch.label
            output = model(text)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            epoch_acc += (output.argmax(1) == labels).sum().item()

            _, predicted = torch.max(output.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        avg_loss = epoch_loss / len(train_iter)
        avg_acc = epoch_acc / len(train_iter.dataset)

        epoch_accuracy = 100 * correct / total

        print(f' Epoch {epoch + 1}, Train Loss: {avg_loss}, Train Accuracy: {epoch_accuracy}')

        # Early Stopping
        val_loss = eval_model(model, val_iter, criterion)
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping")
            break

def eval_model(model, val_iter, criterion):
    model.eval()
    epoch_loss = 0
    epoch_acc = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in val_iter:
            text, labels = batch.text, batch.label
            output = model(text)
            loss = criterion(output, labels)
            epoch_loss += loss.item()
            epoch_acc += (output.argmax(1) == labels).sum().item()
            _, predicted = torch.max(output.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = epoch_loss / len(val_iter)
    avg_acc = epoch_acc / len(val_iter.dataset)

    epoch_accuracy = 100 * avg_acc

    print(f'Validation Loss: {avg_loss},  Validation Accuracy: {epoch_accuracy}')
    return avg_loss

def test_model(model, test_iter, criterion):
    model.eval()
    epoch_loss = 0
    epoch_acc = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in test_iter:
            text, labels = batch.text, batch.label
            output = model(text)
            loss = criterion(output, labels)
            epoch_loss += loss.item()
            epoch_acc += (output.argmax(1) == labels).sum().item()
            _, predicted = torch.max(output.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = epoch_loss / len(test_iter)
    avg_acc = epoch_acc / len(test_iter.dataset)

    epoch_accuracy = 100 * correct / total

    print(f'Test Loss: {avg_loss},  Test Accuracy: {epoch_accuracy}')

# Load data splits
TEXT = data.Field(sequential=True, batch_first=True, lower=True)
LABEL = data.LabelField()
train_data, val_data, test_data = datasets.SST.splits(TEXT, LABEL)

# Build dictionary
TEXT.build_vocab(train_data)
LABEL.build_vocab(train_data)

# Hyperparameters
vocab_size = len(TEXT.vocab)
label_size = len(LABEL.vocab)
padding_idx = TEXT.vocab.stoi['<pad>']
embedding_dim = 300
hidden_dim = 512
dropout = 0.5
num_layers = 2
drop_prob = 0.2

# Build iterators
train_iter, val_iter, test_iter = data.BucketIterator.splits(
    (train_data, val_data, test_data),
    batch_size=64)

torch.manual_seed(42)
model = ResRNN(vocab_size, embedding_dim, hidden_dim, label_size, padding_idx, dropout, num_layers, drop_prob)
optimizer = optim.RMSprop(model.parameters(), lr=0.001, alpha=0.9)
criterion = nn.CrossEntropyLoss()

train_model(model, train_iter, val_iter, optimizer, criterion, num_epochs=20)

# Evaluate the model on validation data
eval_model(model, val_iter, criterion)

# Evaluate the model on test data
test_model(model, test_iter, criterion)


RuntimeError: input.size(-1) must be equal to input_size. Expected 512, got 300